In [ ]:
#cjh t8

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn import decomposition
from imblearn.over_sampling import SMOTE

In [2]:
# 구글 드라이브 연동

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/drive/MyDrive/SPAI/graph_data'
/content


In [10]:
import csv
import glob
import os
import sys

In [49]:
input_path = '/content/gdrive/MyDrive/LAB/data_300/'

In [50]:
import os

filelist = os.listdir(input_path) # 현재 디렉토리의 모든 파일명
fileset = [file for file in filelist if file.endswith('.csv')] # csv 파일명만 추출

len(fileset) # csv 파일 개수 출력 (11218)

300

In [51]:
benign_file = [file for file in fileset if file.startswith('0')]
malicious_file = [file for file in fileset if file.startswith('1')]

benign_cnt = len(benign_file)
malicious_cnt = len(malicious_file)

print(benign_cnt, malicious_cnt) # benign 파일과 malicious 파일 개수 출력

150 150


In [52]:
x  = pd.DataFrame()

for file in benign_file:
    df = pd.read_csv(input_path + file)
    data = [] # columns별 연결된 노드 개수 count하여 저장
    
    for col in df.columns:
        data.append(df[col].sum())
    new_df = pd.DataFrame(columns = df.columns)
    new_df.loc[0] = data
    new_df['class'] = 0
    x = pd.concat([x, new_df])

In [53]:
malicious_loop = malicious_cnt / 50; # malicious 50개씩 끊어서 로드

In [54]:
for turn in range(int(malicious_loop)): 
    for file in malicious_file[turn * 50 : (turn + 1) * 50]:
        df = pd.read_csv(input_path + file)
        data = [] # columns별 연결된 노드 개수 count하여 저장
        
        for col in df.columns:
            data.append(df[col].sum())
            
        new_df = pd.DataFrame(columns = df.columns)
        new_df.loc[0] = data
        new_df['class'] = 1
        x = pd.concat([x, new_df])
    print(turn) # (과정 확인용)

0
1
2


In [55]:
# 남은 malicious 파일 로드
for file in malicious_file[int(malicious_loop)*50 : ]:
    df = pd.read_csv(input_path + file)
    data = [] # columns별 연결된 노드 개수 count하여 저장
        
    for col in df.columns:
        data.append(df[col].sum())
            
    new_df = pd.DataFrame(columns = df.columns)
    new_df.loc[0] = data
    new_df['class'] = 1
    x = pd.concat([x, new_df])

In [56]:
x.fillna(-1, inplace=True) # 결측치 -1로 채우기
x.tail()

,AcroForm,AcroForm.1,DA,DR,Encoding,PDFDocEncoding,PDFDocEncoding.1,Differences,Type,Font,HeBo,HeBo.1,BaseFont,Encoding.1,Name,Subtype,Type.1,Helv,Helv.1,BaseFont.1,Encoding.2,Name.1,Subtype.1,Type.2,ZaDb,ZaDb.1,BaseFont.2,Name.2,Subtype.2,Type.3,Fields,Fields.1,AP,D,Off,Off.1,BBox,Length,Resources,ProcSet,...,CT,FS,TF,AA.1,PV,PV.1,ImageOpacity,ImageOpacity.1,MyFont,BE,BKC,FN,FoxitTag,FoxitTag.1,BoxWidth,CharColor,CharSpace,FontSize,HorzScale,License,LineFeed,MenderFlag,OriginX,OriginY,TextMatrix,bChangeBox,IT,IF,FB,SW,Open,Popup,ZiTi,ZiTi.1,FxTag,3DA,DIS,TB,3DD,3DD.1
0,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [57]:
x.shape

(300, 3652)

In [58]:
x.to_csv("/content/gdrive/MyDrive/LAB/preprocess.csv")

In [61]:
x = pd.read_csv("/content/gdrive/MyDrive/LAB/preprocess.csv") # 전처리 데이터 불러오기
columns = x.columns
columns = columns.drop("class")
columns = columns.drop("Unnamed: 0") # (index_ignore=True 하지 않아서 Unnamed: 0 컬럼 생성 -> 제거)

# 라벨 분리
y = x["class"]
x = x.drop("class", axis = 1)
x = x.drop("Unnamed: 0", axis = 1) # (index_ignore=True 하지 않아서 Unnamed: 0 컬럼 생성 -> 제거)

In [65]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn

In [67]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=99)

In [68]:
print(x_train.shape, x_train.ndim)
print(y_train.shape, y_train.ndim)

(240, 3651) 2
(240,) 1


In [103]:
model = Sequential()

model.add(Conv1D(32, 4, strides = 1, activation = 'relu', input_shape=(3651, 1)))

model.add(Conv1D(64, 4, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_38 (Conv1D)          (None, 3648, 32)          160       
                                                                 
 conv1d_39 (Conv1D)          (None, 3645, 64)          8256      
                                                                 
 dropout_19 (Dropout)        (None, 3645, 64)          0         
                                                                 
 dense_19 (Dense)            (None, 3645, 1)           65        
                                                                 
Total params: 8,481
Trainable params: 8,481
Non-trainable params: 0
_________________________________________________________________


In [105]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
8/8 [==============================] - 3s 292ms/step - loss: 0.6688 - accuracy: 0.5407
Epoch 2/10
8/8 [==============================] - 3s 316ms/step - loss: 0.6650 - accuracy: 0.5916
Epoch 3/10
8/8 [==============================] - 3s 310ms/step - loss: 0.6641 - accuracy: 0.5608
Epoch 4/10
8/8 [==============================] - 2s 296ms/step - loss: 0.6605 - accuracy: 0.5704
Epoch 5/10
8/8 [==============================] - 2s 300ms/step - loss: 0.6573 - accuracy: 0.5908
Epoch 6/10
8/8 [==============================] - 2s 306ms/step - loss: 0.6600 - accuracy: 0.5859
Epoch 7/10
8/8 [==============================] - 3s 324ms/step - loss: 0.6547 - accuracy: 0.5871
Epoch 8/10
8/8 [==============================] - 2s 291ms/step - loss: 0.6561 - accuracy: 0.5879
Epoch 9/10
8/8 [==============================] - 3s 319ms/step - loss: 0.6550 - accuracy: 0.5854
Epoch 10/10
8/8 [==============================] - 3s 314ms/step - loss: 0.6565 - accuracy: 0.5844
